In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms

import cv2
import numpy as np
from torchvision import transforms
from PIL import Image

In [18]:
#Класс для создания датасета с увеличением количества примеров в 2 раза
class DatasetWithAugmentations(Dataset):
    def __init__(self, root, transform):
        self.dataset = datasets.ImageFolder(root=root)
        self.transform = transform

    def __len__(self):
        return len(self.dataset) * 2  # Удваиваем количество изображений

    def __getitem__(self, idx):
        original_idx = idx // 2  # Определяем индекс исходного изображения
        image, label = self.dataset[original_idx]

        # Применяем аугментацию дважды
        augmented_image1 = self.transform(image)
        augmented_image2 = self.transform(image)

        if idx % 2 == 0:
            return augmented_image1, label
        else:
            return augmented_image2, label

In [19]:
def apply_clahe(image):
    # Преобразуем изображение из PIL в numpy array
    image_np = np.array(image)

    # Применяем CLAHE к каждому каналу (RGB)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image_np[:, :, 0] = clahe.apply(image_np[:, :, 0])
    image_np[:, :, 1] = clahe.apply(image_np[:, :, 1])
    image_np[:, :, 2] = clahe.apply(image_np[:, :, 2])

    # Преобразуем обратно в PIL Image
    image_pil = Image.fromarray(image_np)
    return image_pil


def transform(with_clahe, with_flip):
    layers = []
    # Определяем преобразования для изображений
    if with_clahe:
        layers.append(transforms.Lambda(apply_clahe))
    layers.append(transforms.Resize((480, 640)))
    if with_flip:
        layers.append(transforms.RandomHorizontalFlip(p=0.5))
        layers.append(transforms.RandomVerticalFlip(p=0.5))
    # if with_rotation:
    #     transforms.RandomRotation(degrees=(-90, 90)), # Пока убрал так как проблема с обрезанием картинки не решена
    layers.append(transforms.ToTensor())
    return transforms.Compose(layers)

In [20]:
# функция для создания увеличенного в 2 раза датасета с аугментациями
def CreateDatasetWithAugmentations(root, with_clahe, with_flip):
    return DatasetWithAugmentations(
        root=root,
        transform=transform(with_clahe, with_flip)
    )